# TODO:
### update frsq poi mapping
### assigning rltn tag to equal ln may not be a good idea. 

for exmaple, when the role of a way is inner(http://www.openstreetmap.org/way/48076826), meaning the the way serve a different function than rltn(http://www.openstreetmap.org/relation/381363)

it seems that "inner" role is the only fallback

examples:
- set(['cycling facilities', 'retail shop']) http://www.openstreetmap.org/way/48076826 set(['transportation']) http://www.openstreetmap.org/relation/381363
- set(['transportation']) http://www.openstreetmap.org/way/48081175 set(['outdoors and recreation']) http://www.openstreetmap.org/relation/3213622
- set(['professional service']) http://www.openstreetmap.org/way/48085321 set(['transportation']) http://www.openstreetmap.org/relation/914244
- set(['cycling facilities']) http://www.openstreetmap.org/way/48168497 set(['transportation']) http://www.openstreetmap.org/relation/383188
- set(['cycling facilities']) http://www.openstreetmap.org/way/52303179 set(['transportation']) http://www.openstreetmap.org/relation/450333
- set(['professional service']) http://www.openstreetmap.org/way/66417858 set(['outdoors and recreation']) http://www.openstreetmap.org/relation/3213622
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/173918307 set(['cycling facilities']) http://www.openstreetmap.org/relation/2322204
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/66418767 set(['cycling facilities']) http://www.openstreetmap.org/relation/2322204
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/237887425 set(['cycling facilities']) http://www.openstreetmap.org/relation/3205766
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/237887426 set(['cycling facilities']) http://www.openstreetmap.org/relation/3205766
- set(['cycling facilities']) http://www.openstreetmap.org/way/203025090 set(['transportation']) http://www.openstreetmap.org/relation/2724481
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170966
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170968
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170967
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170968
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170966
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170967
- set(['cycling facilities']) http://www.openstreetmap.org/way/296755159 set(['transportation']) http://www.openstreetmap.org/relation/2723927
- set(['cycling facilities']) http://www.openstreetmap.org/way/296794684 set(['transportation']) http://www.openstreetmap.org/relation/2723927

In [37]:
import src.constants; reload(src.constants)
from src.constants import fn_frsq_venues_dc, fn_segments_dc, epsg_dc, fn_osm_db_dc, fn_poi_frsq_dc, fn_poi_osm_dc
from src.constants import fn_mapping_for_fs, fn_mapping_for_osm, poi_categories

import src.geom_helper; reload(src.geom_helper)
from src.geom_helper import bfr_20m, objs_near_segs, objs_near_segs_store, gpdf_equal

import geopandas as gp
import pandas as pd
import numpy as np

import src.poi; reload(src.poi)
from src.poi import *

# TODO

1. [x] mapped frsq venues to poi categories --> filter frsq venues by obj_near_segment
2. [x] mapped osm data to poi categories, filter osm data by obj_near_segment
3. [x] remove overlap between osm and frsq
4. [-] Visalization:
    - [x] poi categories distribution
        1. [x] frsq venues near segments
        2. [x] osm venues near segments
        3. [x] final poi near segments(after removing overlap)
    - [] poi categories spatial distribution

In [2]:
data_dir = 'data/'
path_frsq_venues_dc = data_dir + fn_frsq_venues_dc
path_osm_db_dc = data_dir + fn_osm_db_dc
path_segments_dc = data_dir + fn_segments_dc
path_mapping_for_fs = data_dir+fn_mapping_for_fs
path_mapping_for_osm = data_dir+fn_mapping_for_osm


In [ ]:
def poi_in_city(path_frsq, path_osm_db, path_segs, path_mapping_frsq, path_mapping_osm, debug=False):
    frsq_venues = gp.read_file(path_frsq)
    segs = gp.read_file(path_segs)
    poi_frsq = map_frsq_venues_to_poi_category(frsq_venues, )
    print '# venues in city=',poi_frsq_dc.shape[0]
    
    

In [3]:
frsq_venues_dc = gp.read_file(path_frsq_venues_dc)
segments_dc = gp.read_file(path_segments_dc)
poi_frsq_dc = map_frsq_venues_to_poi_category(frsq_venues_dc, path_mapping_for_fs, debug=True)
print '# venues in city=',poi_frsq_dc.shape[0]

===========mapping  frsq venues to poi categories===========
venues without poi category: #venues=3139, #frsq_categories=63
top ten frsq_categories {u'': 1447, u'Building': 882, u'TV Station': 11, u'Exhibit': 51, u'Moving Target': 121, u'Zoo': 14, u'Festival': 6, u'Conference': 7, u'Intersection': 55, u'Road': 450}
# venues in city= 30187


In [4]:
poi_osm_dc = map_osm_to_poi_category(path_osm_db_dc, path_mapping_for_osm, debug=True)
poi_osm_pts_dc, poi_osm_ln_pls_dc = clean_and_split_poi_osm_by_geometry(poi_osm_dc)


===========mapping osm to poi category===========
# rows in table tag  = 1484405
# rows after mapping = 44060
# objs have category = 32808
# objs have category by ot: {'R': 1079, 'W': 20010, 'N': 11719}
# categories: # objs = {1: 32183, 2: 620, 3: 5}


d:\program files\anaconda2\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
poi_pts_dc = remove_osm_frsq_overlap(poi_osm_pts_dc, poi_frsq_dc, path_osm_db_dc, epsg_dc)

=====remove_osm_frsq_overlap=====
# conn components(osm_pts connected with frsq):  2727
# pairs of overlap (891, 13)
# pairs by overlap criteria {'same category': 111, 'similar name': 780}
distribution of # pairs of overlap within each connected component {1: 681, 2: 70, 3: 12, 4: 6, 5: 2}
# overlap points in osm = 857, # points in fs = 873
# independent points in osm = 10906, # points in fs = 29314
# total points in osm = 11725, # points in fs = 30187


In [6]:
poi_dc = poi_osm_ln_pls_dc.append(poi_pts_dc, ignore_index=True)

In [7]:
from src.geom_helper import crs_prepossess

poi_frsq_dc_bfr = crs_prepossess(poi_frsq_dc, init_crs=4326, bfr_crs=epsg_dc)
poi_osm_dc_bfr = crs_prepossess(poi_osm_dc, init_crs=4326, bfr_crs=epsg_dc)

poi_frsq_dc_bfr.geometry = poi_frsq_dc_bfr.buffer(5)
poi_osm_dc_bfr.geometry = poi_osm_dc_bfr.buffer(5)
poi_frsq_dc_bfr = poi_frsq_dc_bfr.to_crs(epsg=4326)
poi_osm_dc_bfr = poi_osm_dc_bfr.to_crs(epsg=4326)

poi_frsq_near_seg_dc = objs_near_segs(poi_frsq_dc_bfr, segments_dc, bfr_20m, epsg_dc, output='objs')
poi_osm_near_seg_dc = objs_near_segs(poi_osm_dc_bfr, segments_dc, bfr_20m, epsg_dc, output='objs')


poi_near_seg_dc = objs_near_segs(poi_dc, segments_dc, bfr_20m, epsg_dc, output='objs')

# print '# venues near segments=', poi_frsq_near_seg_dc.shape[0]
# objs_near_segs_store(poi_frsq_near_seg_dc, data_dir, fn_poi_frsq_dc, fn_segments_dc)
# print '# objs have category near seg by ot:', poi_osm_near_seg_dc.ot.value_counts().to_dict()
# temp = poi_osm_near_seg_dc.copy()
# temp.category = temp.category.apply(lambda x: str(x))
# objs_near_segs_store(temp, data_dir, fn_poi_osm_dc, fn_segments_dc)
# del temp

In [8]:
print '# poi in dc = {}, # poi near seg dc = {}'.format(poi_dc.shape[0], poi_near_seg_dc.shape[0])
print '# objs have category near seg by ot:', poi_near_seg_dc.ot.value_counts().to_dict()

# poi in dc = 64484, # poi near seg dc = 44668
# objs have category near seg by ot: {'R': 1398, 'N_fq': 729, 'W': 14389, 'N': 8081}


In [9]:
poi_distribution(poi_frsq_near_seg_dc, poi_osm_near_seg_dc,poi_near_seg_dc)

,category,fs,osm,poi,fo
0,professional service,4442,998,5373,5440
1,food,3132,1173,4065,4305
2,retail shop,2950,1342,4170,4292
3,outdoors and recreation,2590,3931,6418,6521
4,transportation,1489,5664,7134,7153
5,residence,1236,537,1741,1773
6,nightlife spot,1190,203,1352,1393
7,schools&university,827,455,1270,1282
8,art,606,72,672,678
9,cycling facilities,153,10865,10905,11018


In [12]:
seg_poi_index, poi_near_seg_dc = objs_near_segs(poi_dc, segments_dc, bfr_20m, epsg_dc, output='index_and_objs')

In [24]:
seg_poi_index.merge(segments_dc, left_on='seg_index', right_index=True).head()

,seg_index,obj_index,BEGINMEASURE,DIRECTIONALITY,ENDMEASURE,FACILITYID,FROMADDRESSLEFTTHEO,FROMADDRESSRIGHTTHEO,FROMNODEID,OBJECTID,...,SHAPE_Length,SOURCEID,STREETID,STREETSEGID,STREETTYPE,TOADDRESSLEFTTHEO,TOADDRESSRIGHTTHEO,TONODEID,UPDATETIMESTAMP,geometry
0,0,27568,0,0,192.925133,SEGID-876,5000,5001,18637,1,...,192.925131,10010500,100010,876,ST,5098,5099,22102,2005-11-01T12:55:46,LINESTRING (-77.01239193659748 38.950836102255...
1,0,50299,0,0,192.925133,SEGID-876,5000,5001,18637,1,...,192.925131,10010500,100010,876,ST,5098,5099,22102,2005-11-01T12:55:46,LINESTRING (-77.01239193659748 38.950836102255...
2,0,536,0,0,192.925133,SEGID-876,5000,5001,18637,1,...,192.925131,10010500,100010,876,ST,5098,5099,22102,2005-11-01T12:55:46,LINESTRING (-77.01239193659748 38.950836102255...
3,1,32698,0,1,131.935650,SEGID-13161,5200,5203,20290,2,...,131.935677,10010520,100010,13161,ST,5298,5299,19814,2005-11-01T12:55:46,LINESTRING (-77.01163764014346 38.953383268855...
4,1,49971,0,1,131.935650,SEGID-13161,5200,5203,20290,2,...,131.935677,10010520,100010,13161,ST,5298,5299,19814,2005-11-01T12:55:46,LINESTRING (-77.01163764014346 38.953383268855...


In [21]:
def get_sum_categories(list_mapped):
    from collections import defaultdict
    sum_categories = defaultdict(int)
    for mapped in list_mapped:
        for c in list(mapped):
            sum_categories[c]+=1
    return sum_categories

In [27]:
seg_poi_index_category = seg_poi_index.merge(poi_near_seg_dc[['category']], left_on='obj_index', right_index=True)

In [29]:
poi_per_seg = {}
for idx, grp in seg_poi_index_category.groupby('seg_index'):
    poi_per_seg[idx] = dict(get_sum_categories(grp.category.values))


In [30]:
len(poi_per_seg), segments_dc.shape

(12530, (13522, 22))

In [33]:
poi_per_seg = pd.DataFrame(poi_per_seg.items(),columns=['seg_index','category'])

In [34]:
poi_per_seg['total'] = poi_per_seg.category.apply(lambda x: sum(x.values()) if type(x)!=float else 0)

In [38]:
for l in poi_categories:
    poi_per_seg[l] = poi_per_seg.category.apply(lambda x: x.get(l,0) if type(x)!=float else 0)

In [40]:
poi_per_seg.drop('category',axis=1,inplace=True)